In [1]:
from PIL import Image
from os import path, makedirs
import os
import re
import pandas as pd
import sys
import argparse

In [2]:
# labels = multi_df["class"].unique()
def convert_csv_to_yolo(multi_df, path, target_name):

    # Encode labels according to labeldict if code's don't exist
    if not "code" in multi_df.columns:
        multi_df["code"] = multi_df["class"].apply(lambda x: labeldict[x])
    # Round float to ints
    for col in multi_df[["xmin", "ymin", "xmax", "ymax"]]:
        multi_df[col] = (multi_df[col]).apply(lambda x: round(x))

    # Create Yolo Text file
    last_image =""
    txt_file = ""

    for index, row in multi_df.iterrows():
        if not last_image == row["filename"]:
            txt_file += "\n" + path + row["filename"] + " "
            txt_file += ",".join(
                [
                    str(x)
                    for x in (row[["xmin", "ymin", "xmax", "ymax", "code"]].tolist())
                ]
            )
        else:
            txt_file += " "
            txt_file += ",".join(
                [
                    str(x)
                    for x in (row[["xmin", "ymin", "xmax", "ymax", "code"]].tolist())
                ]
            )
        last_image = row["filename"]
    file = open(target_name, "w")
    file.write(txt_file[1:])
    file.close()
    return True

In [6]:
labeldict=dict(zip(["raccoon"],[0,]))
train_csv_path = './CSV_train_labels.csv'
multi_df = pd.read_csv(train_csv_path)
convert_csv_to_yolo(multi_df, path="./images", target_name="data_train.txt")

test_csv_path = './CSV_test_labels.csv'
multi_df = pd.read_csv(test_csv_path)
convert_csv_to_yolo(multi_df, path="./images", target_name="data_test.txt")

True